# Scraping Rosters and Player Headshots

Sources: 

* Rosters: [NHL API](https://statsapi.web.nhl.com/api/v1/teams/3?expand=team.roster)
* Headshots: [Artemi Panarin](https://cms.nhl.bamgrid.com/images/headshots/current/168x168/8478550.jpg)

In [1]:
import requests
import json
import pandas as pd
import os

## New York Rangers Only

##### Grab Roster IDs

In [2]:
# get all team data
team_id = 3
r = requests.get(url=f'https://statsapi.web.nhl.com/api/v1/teams/{team_id}?expand=team.roster')
d = r.json()
roster = d['teams'][0]['roster']['roster']

In [3]:
master_roster = {}

In [4]:
player_info = {}
for player in roster:
    player_info[player['person']['id']] = {
        'currentTeamId': team_id,
        'name': player['person']['fullName'],
        'jersey': player['jerseyNumber'],
        'position': player['position']['abbreviation']
    }

# update rosters
with open('data/roster.json') as f:
    master_roster = json.load(f)
master_roster.update(player_info)
    
with open('data/roster.json', 'w', encoding='utf-8') as f:
    json.dump(master_roster, f, ensure_ascii=False, indent=4)

##### Download Images for Each Player

In [9]:
team_id = 3

# create path for team folder if it doesn't exist
if not os.path.exists(f'data/{team_id}'):
    os.makedirs(f'data/{team_id}/headshots')
    
    
# get player images, download to team folder
for player_id, val in player_info.items():
    link = f'https://cms.nhl.bamgrid.com/images/headshots/current/168x168/{player_id}.jpg'
    img = requests.get(link).content
    with open(f'data/{team_id}/headshots/{player_id}.jpg', 'wb') as handler:
        handler.write(img)